In [ ]:
import uproot

# Input and output ROOT files
input_file = "/home/users/dprimosc/HH4b/src/HH4b/overlap/eventlist_files_2024Oct22/eventlist_boostedHH4b_2023BPix.root"
output_file = "/home/users/dprimosc/HH4b/src/HH4b/overlap/eventlist_files_2024Oct22_cleaned/eventlist_boostedHH4b_2023BPix.root"

# Open the input file
file = uproot.open(input_file)

# Create a new ROOT file for writing
with uproot.recreate(output_file) as new_file:
    for key in file.keys():
        name, version = key.split(";")
        version = int(version)

        if version == 1:
            print(f"Skipping {name};1 (will be replaced by {name};2)")
            continue  # Skip old versions

        elif version == 2:
            # Rename the TTree to ;1
            new_name = name  # Drop the ";2" and keep it as the base name
            print(f"Renaming {name};2 to {new_name};1")
            tree = file[key]
            # Write the tree to the new file
            new_file[new_name] = tree.arrays(library="np")

        else:
            # Write any other objects as-is
            print(f"Copying {name};{version} without modification")
            tree = file[key]
            new_file[name] = tree.arrays(library="np")

print(f"Updated ROOT file saved as {output_file}")

In [ ]:
# Verification Step
print("\nStarting verification process...")

# Open both files for comparison
original_file = uproot.open(input_file)
updated_file = uproot.open(output_file)

for key in original_file.keys():
    name, version = key.split(";")
    if version == "1" and f"{name};2" in original_file.keys():
        # Skip checking tree-name;1 as it's replaced
        continue

    # Get data from both files
    original_data = original_file[key].arrays(library="np")
    updated_data = updated_file[name].arrays(library="np")

    # Compare branches in the TTree
    mismatches = False
    for branch in original_data:
        if branch not in updated_data:
            print(f"Branch {branch} missing in updated data for {key}.")
            mismatches = True
        elif not (original_data[branch] == updated_data[branch]).all():
            print(f"Mismatch in branch {branch} for {key}.")
            mismatches = True

    if not mismatches:
        print(f"Data verified for {key}.")

print("Verification process completed.")

In [ ]:
# Print contents of the 'hh4b' tree
if "hh4b" in file:
    print("\nContents of 'hh4b' tree:")
    hh4b_tree = file["hh4b"]
    max_entries = 50  # Maximum number of entries to display

    # Display the list of branches
    print("Branches:")
    for branch in hh4b_tree.keys():
        print(f"  - {branch}")

    # Display the first few rows of data for each branch
    print("\nSample data from 'hh4b':")
    hh4b_data = hh4b_tree.arrays(library="np")
    for branch, data in hh4b_data.items():
        print(f"{branch}: {data[:max_entries]}")  # Print the first 5 entries for each branch
else:
    print("\n'hh4b' tree not found in the file.")